# Automates the reading and stacking of Sentinel-2 Imagery

I use this code after I download S2 imagery from Copernicus. (https://scihub.copernicus.eu/dhus/#/home)

Stacks only the visible bands, NIR, RE and SWIR1 and SWIR2 (bands 2, 3, 4, 8, 8A, 11, 12). Resamples 20m bands (8A, 11 and 12) to 10m. This can be edited to include additional bands, but you will need to change the code.

Parameters:
dir_list: list of string, Create a list of image directories for every image you want processed. Set the directory to the 'IMG_DATA' file. Sentinel-2 L2A should all be formatted the same, which lets this code be universal

WINDOWS USERS: You will need to change the "/" to "\" where indicated. The direction of the slash matters or the code won't work for you.

In [1]:
import os
import numpy as np
import numpy.ma as ma
import rasterio
from rasterio.enums import Resampling

In [2]:
### This is the only block you will need to edit ###

dir_list = ['.../IMG_DATA',
           ]

In [3]:
def untile_image(img_fp):
    '''
    Removed tiling from image to allow for resamling to match imagery
    '''
    img = rasterio.open(img_fp)
    img_array = img.read(1)    
    untiled_fp = '%s_untiled.tif' %img_fp[:-4]

    with rasterio.Env():
        profile = img.profile
        profile['tiled']=False
        profile['driver']='GTiff'
        with rasterio.open(untiled_fp, 'w', **profile) as dst:
            dst.write(img_array, 1)

    return untiled_fp

def rescale (scale_factor, dataset, ref_dataset, output_fp):
    
    scale_factor = scale_factor

    with rasterio.open(dataset) as dataset:

        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.bilinear 
        )
        
    ref_img = rasterio.open(ref_dataset)

    with rasterio.Env():
        profile = ref_img.profile
        profile['tiled'] = False 
        profile['driver'] = 'GTiff'
        with rasterio.open(output_fp, 'w', **profile) as dst:
            dst.write(data)
            
def get_image_name(image_directory):
    file = fp + '/R10m'              #WINDOWS USERS: change slash direction here.
    names = os.listdir(file)[0]
    img_name = names[:22]
    return img_name

In [4]:
# WINDOWS USERS:  Change the direction of the slash where needed in
#                 all the strings in this block.

for fp in dir_list:
    os.chdir(fp)
    
    img_identifier = get_image_name(fp)

    re = 'R20m/%s_B8A_20m.jp2' %img_identifier 
    swir1 = 'R20m/%s_B11_20m.jp2' %img_identifier 
    swir2 = 'R20m/%s_B12_20m.jp2' %img_identifier

    ref_img = 'R10m/%s_B02_10m.jp2' %img_identifier 
    
    stack_list = ['R10m/%s_B02_10m.jp2' %img_identifier,
                  'R10m/%s_B03_10m.jp2' %img_identifier,
                  'R10m/%s_B04_10m.jp2' %img_identifier,
                  'R10m/%s_B08_10m.jp2' %img_identifier]
    
    untiled_10m_images = []
    
    for image in stack_list:
        img = rasterio.open(image)
        if img.profile['tiled'] == True:
            untiled_img_fp = untile_image(image)
            untiled_10m_images.append(untiled_img_fp)
            
    if len(untiled_10m_images) != 0:
        stack_list = untiled_10m_images
    
    #Set up a list 20m images to be incorperated into the stack
    images_20m = [re, swir1, swir2]
    
    # Determine if image needs to be untiled
    untiled_images = []
    
    for image in images_20m:
        img = rasterio.open(image)
        if img.profile['tiled'] == True:
            untiled_img_fp = untile_image(image)
            untiled_images.append(untiled_img_fp)
            
    if len(untiled_images) != 0:
        images_20m = untiled_images
    
    # Resample 20m images to 10m and add them to the list to be stacked
    for image in images_20m:
        rescaled_fp = image[:-4] + '_resampled.tif'
        stack_list.append(rescaled_fp)
        
        rescale((2), image, ref_img, rescaled_fp)
    
    # Open all bands and write to new file
    # Read metadata of first file
    with rasterio.open(stack_list[0]) as src0:
        meta = src0.meta

    # Update meta to reflect the number of layers
    meta['count'] = len(stack_list)
    meta['nodata'] = 0
    meta['tiled'] = True
    
    new_stack = '%s_stack.tif' %img_identifier
    
    for num, image in enumerate(stack_list):
        print('Writing band ', num+1)
        org = rasterio.open(image)
        arr = org.read(1)
        
        if num == 0:
            with rasterio.open(new_stack, 'w', **meta) as dst:
                dst.write(arr, num+1)
        else:
            with rasterio.open(new_stack, 'r+', **meta) as dst:
                dst.write(arr, num+1)

Writing band  1
Writing band  2
Writing band  3
Writing band  4
Writing band  5
Writing band  6
Writing band  7
Writing band  1
Writing band  2
Writing band  3
Writing band  4
Writing band  5
Writing band  6
Writing band  7
Writing band  1
Writing band  2
Writing band  3
Writing band  4
Writing band  5
Writing band  6
Writing band  7
